In [4]:
import psycopg2
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
import pandas as pd
import time
import re
import numpy as np
from config import host, user, user1, password, password1

In [5]:
postgres_engine = create_engine(f'postgresql://{user1}:{password1}@{host}:5432/staging')

In [6]:
#PostgreSQL Connection
try:
    pg_db = psycopg2.connect(host=host,user=user1,password=password1,database="staging")
    pg_db.autocommit = True
    pgcursor = pg_db.cursor()
    print("PostgreSQL Connection Established")
except psycopg2.OperationalError as e:
    print("Unable to Connect: ",format(e))

PostgreSQL Connection Established


In [ ]:
stg_crm = pd.read_sql('SELECT * FROM stg_crm_customer', con=postgres_engine)
stg_bpm = pd.read_sql('SELECT * FROM stg_bpm_client_info', con=postgres_engine)
stg_blc = pd.read_sql('SELECT * FROM stg_blc_customer', con=postgres_engine)
master = pd.read_sql('SELECT * FROM master_customer_test', con=postgres_engine)

In [ ]:
master.columns

In [ ]:
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

### CRM

In [ ]:
# crm["source_table"] = "CRM_Customer"
stg_crm["role_id"] = 1
stg_crm = stg_crm.rename(columns={"locality": "city", "region": "state", "country_code": "country", 
                          "given_name": "first_name", "phonenumber": "phone", "address1": "address_line1", 
                          "address2": "address_line2", "zip_code": "zip"})
stg_crm = stg_crm[['customer_id', 'email_address', 'date_created', 'first_name','country', 'phone', 'email_status', 
           'created_date', 'updated_date','role_id']]#,'source_table']]
stg_crm.shape

In [ ]:
#Validating Email_addresses Using REGEX
crm = stg_crm[stg_crm.email_address.str.contains('[\w\.-]+@[\w\.-]+', regex= True, na=False)]
crm

In [ ]:
stg_crm[~stg_crm.isin(crm)].dropna()
# .shape = 25

In [ ]:
new_master = master.append(crm)
# new_master

### BPM

In [ ]:
stg_bpm.columns

In [ ]:
# bpm["source_table"] = "BPM_Client_INFO"
stg_bpm["role_id"] = 1
stg_bpm = stg_bpm.rename(columns={"company_name": "publisher_name", "email": "email_address", "address": "address_line1", "date": "date_of_sale", "clientid": "customer_id"})
stg_bpm = stg_bpm[['customer_id', 'publisher_name', 'address_line1', 'city', 'state','country', 'zip', 'phone', 'fax', 
           'email_address','date_of_sale', 'created_date', 'updated_date','role_id']]#,'source_table']]
stg_bpm.shape

In [ ]:
#Validating Email_addresses Using REGEX
bpm = stg_bpm[stg_bpm.email_address.str.contains('[\w\.-]+@[\w\.-]+', regex= True, na=False)]
bpm

In [ ]:
# result = pd.merge(stg_bpm,bpm,on='customer_id',how='outer',indicator=True)
# result=result[result['_merge']=='left_only']
# print(result.shape)
# result

In [ ]:
# from pandas import ExcelWriter

In [ ]:
# writer = ExcelWriter('MissingEmailBPM.xlsx')
# result.to_excel(writer,'Sheet1')
# writer.save()

In [ ]:
# one[one.isin(result)]
# from IPython.display import display

In [ ]:
# for i in result['publisher_name_x'].unique():
#     column_specific = stg_bpm[(stg_bpm['publisher_name']==i)]
# #     if column_specific.shape[0] > 1:
# #         display(column_specific)
        
# #         # Howard Shapiro
# #         # Colleen Sen
        
# #     else:
#     display(column_specific)

In [ ]:
new_master = new_master.append(bpm)
new_master

### BLC

In [ ]:
stg_blc.columns

In [ ]:
stg_blc["source_table"] = "BLC_Customer_Updated"
stg_blc = stg_blc[['customer_id', 'date_created','deactivated', 'email_address','first_name', 'last_name', 'password', 
           'user_name', 'address_line1', 'city', 'phone', 'zip','country_name', 'state_name', 'role_id',
           'is_publisher', 'publisher_name', 'website', 'is_partner', 'date_of_sale','lead_owner', 
           'lead_source', 'order_type', 'created_date','updated_date']]
stg_blc = stg_blc.rename(columns={"country_name": "country", "state_name": "state"})
stg_blc = stg_blc.loc[(stg_blc["role_id"] == 1) & pd.notnull(stg_blc["address_line1"])]
stg_blc = stg_blc.fillna('1')

stg_blc.shape

In [ ]:
#Validating Email_addresses Using REGEX
blc = stg_blc[stg_blc.email_address.str.contains('[\w\.-]+@[\w\.-]+', regex= True, na=False)]
blc

In [ ]:
# blc[blc.isin(x)].dropna()

In [ ]:
new_master = new_master.append(blc)
# new_master

### MASTER CUSTOMER TABLE AND GOLDEN RECORD CREATION

In [ ]:
new_master['email_address'] = new_master['email_address'].str.lower()
new_master = new_master.replace({"\'":"''" }, regex=True)
new_master = new_master.replace(r'^([\s])$|^(?![\s\S])$', np.nan, regex=True)
new_master

In [ ]:
len(new_master['email_address'].unique())

In [ ]:
for i in new_master['email_address'].unique():
    print(i)
    
# ### FIRST_NAME
#     # Creating a new dataframe using rows that have email_address = i
#     column_specific = new_master[(new_master['email_address']==i) & (new_master['first_name'].isna()==False)]
# #     print(column_specific)
    
#     if column_specific.shape[0] == 0:
#         first_name = np.nan
#         print("FIRST_NAME:",first_name,'\n')
        
#     elif column_specific.shape[0] == 1:
        
#         first_name = column_specific.iloc[0].last_name
#         print("FIRST_NAME:",first_name,'\n')
        
#     else:
#         # Extract first names to filter out based on frequency for golden record
#         fn = column_specific['first_name'].tolist()

#         # Get frequency counts of each of the names
#         first_names = {x:fn.count(x) for x in fn}
#     #     print("first_name:",first_names)

#         # Checking for ties in frequencies by finding unique counts values
#         first_name_set = set(val for val in first_names.values())
#     #     print("First Name Counts:",first_name_set)

#         # if lengths of dictionary and set are equal = no ties between names = get value with highest frequency
#         # if lengths are not equal = frequencies tied between two names = get the oldest name
#         if len(first_name_set) == len(first_names):
#             first_name = max(first_names, key=first_names.get)
#             print("FIRST_NAME:",first_name,'\n')

#     #         print("---------")
#         else:
#             date_created = min(column_specific['date_created'].tolist())
#     #         print(date_created)

#             idx = column_specific[column_specific['date_created']==date_created].index
#             first_name = list(column_specific.loc[idx,'first_name'])[0]
#             print("FIRST_NAME:",first_name,'\n')

#     #         print("---------")
        
# ### LAST_NAME    
#     # extracting those rows that have email_addresses = i
#     column_specific = new_master[(new_master['email_address']==i) & (new_master['last_name'].isna()==False)]
# #     print(column_specific)
    
#     if column_specific.shape[0] == 0:
#         last_name = np.nan
#         print('last_name:',last_name,'\n')
        
#     elif column_specific.shape[0] == 1:
        
#         last_name = column_specific.iloc[0].last_name
#         print('last_name:',last_name,'\n')
        
#     else:

#         # extract values to filter out based on frequency for golden record
#         ln = column_specific['last_name'].tolist()
#     #     print(ln)

#         #Get frequency counts of each last name in list
#         last_names = {x:ln.count(x) for x in ln}
#     #     print("last name:",last_names)

#         # Checking for ties in frequencies by finding unique counts values
#         last_name_set = set(val for val in last_names.values())
#     #     print("last_name_set:",last_name_set)

#         # if lengths of dictionary and set are equal = no ties between names = get value with highest frequency
#         # if lengths are not equal = frequencies tied between two names = get the oldest name
#         if len(last_name_set) == len(last_names):
#             last_name = max(last_names, key=last_names.get)
#             print('last_name:',last_name,'\n')

#     #         print("---------")
#         else:
#             date_created = min(column_specific['date_created'].tolist())
#     #         print(date_created)

#             idx = column_specific[column_specific['date_created']==date_created].index
#     #         print(idx)
#             last_name = list(column_specific.loc[idx,'last_name'])[0]
#             print('last_name:',last_name,'\n')

#     #         print("---------")


### ADDRESS

    # extracting address columns and rows that satisfy two conditions: 
    # email_addresses = i and address_line1 is not NaN
    column_specific = new_master[['date_created','address_line1','address_line2','city','state','country','zip']][(new_master['email_address']==i) & (new_master['address_line1'].isna()==False)]
#     print(column_specific,'\n')
    
    if column_specific.shape[0] == 0:
        adl1 = adl2 = city = state = country = zip = np.nan
#         print('Address_line1:',adl1)
#         print('Address_line2:',adl2)
#         print('city:',city)
#         print('state:',state)
#         print('Country:',country)
#         print('zip:',zip,'\n')
        
    elif column_specific.shape[0] == 1:
        adl1 = column_specific.iloc[0].address_line1
        adl2 = column_specific.iloc[0].address_line2
        city = column_specific.iloc[0].city
        state = column_specific.iloc[0].state
        country = column_specific.iloc[0].country
        zip = column_specific.iloc[0].zip
#         print('Address_line1:',adl1)
#         print('Address_line2:',adl2)
#         print('city:',city)
#         print('state:',state)
#         print('Country:',country)
#         print('zip:',zip,'\n')
#         print('------------')
        
    else:
        # extract addresses to filter out based on frequency for golden record
        address_lst = column_specific['address_line1'].tolist()
#         print("address_lst:",address_lst,'\n')

        #Frequency counts of each address in list
        address_counts = {x:address_lst.count(x) for x in address_lst}
        
#         print("address_counts:",address_counts,":: set counts:",set(address_counts.values()),'\n')

        # A set cannot have duplicate values, it takes the unique of dictionary values. 
        # Length of set = 1 only when all values in dictionary are the same [Ties might exist]
        if (len(address_counts) > len(set(address_counts.values()))) \
                & (sum(x==max(address_counts.values()) for x in address_counts.values()) == 1):
            
            # Getting most frequent value of column
            freq_value = max(address_counts, key=address_counts.get)
#             print('freq_value:',freq_value,'\n')
            
            # Extracting rows that contain the most frequent value
            freq_df = column_specific[column_specific['address_line1'] == freq_value]
            
            adl1 = freq_df.iloc[0].address_line1
            adl2 = freq_df.iloc[0].address_line2
            city = freq_df.iloc[0].city
            state = freq_df.iloc[0].state
            country = freq_df.iloc[0].country
            zip = freq_df.iloc[0].zip
#             print('Address_line1:',adl1)
#             print('Address_line2:',adl2)
#             print('city:',city)
#             print('state:',state)
#             print('Country:',country)
#             print('zip:',zip,'\n')
        
#             print('FINAL FREQ RECORD')
#             print(freq_df,'\n')

#             print('-----------')
#             print('-----------')
        else:
            
            # Selecting rows with minimum missing values
            # Count number of columns that have max non-null values 
#             count = max(column_specific.count(axis=1))
#             print("count:",count,'\n')
            
            # Get rows with maximum number of non-null values
            min_df = column_specific.dropna(thresh=max(column_specific.count(axis=1)))
            min_df['date_created'] = pd.to_datetime(min_df['date_created'],errors = 'coerce')
#             print('min_df')
#             print(min_df,'\n')

            if (min_df.shape[0]==1) or (min_df['date_created'].isnull().values.any() == True):
                adl1 = min_df.iloc[0].address_line1
                adl2 = min_df.iloc[0].address_line2
                city = min_df.iloc[0].city
                state = min_df.iloc[0].state
                country = min_df.iloc[0].country
                zip = min_df.iloc[0].zip
                print('Address_line1:',adl1)
                print('Address_line2:',adl2)
                print('city:',city)
                print('state:',state)
                print('Country:',country)
                print('zip:',zip,'\n')
                
            else:
                
                # Extract the most recent record of the above dataframe
#                 print(min_df)
                
                date_created = max(min_df['date_created'].tolist())
#                 print('date_created:',type(date_created))
#                 print('date_created:',date_created)
            
                recent_record = min_df[min_df['date_created']==date_created]#.iloc[:,1:]#.values.tolist()[0]
#                 print('recent_record:',recent_record)

                adl1 = recent_record.iloc[0].address_line1
                adl2 = recent_record.iloc[0].address_line1
                city = recent_record.iloc[0].city
                state = recent_record.iloc[0].state
                country = recent_record.iloc[0].country
                zip = recent_record.iloc[0].zip
#                 print('Address_line1:',adl1)
#                 print('Address_line2:',adl2)
#                 print('city:',city)
#                 print('state:',state)
#                 print('Country:',country)
#                 print('zip:',zip,'\n')
            
#             print('-----------')
#             print('-----------')

# ### PHONE

#     # extracting address columns and rows that satisfy two conditions: 
#     # email_addresses = i and address_line1 is not NaN
#     column_specific = new_master[['date_created','phone']][(new_master['email_address']==i) & (new_master['phone'].isna()==False)]
#     column_specific['phone'] = column_specific['phone'].str.replace('\W', '')
# #     print(column_specific,'\n')
    
#     if column_specific.shape[0] == 0:
#         phone_number = np.nan
#         print('PHONE_NUMBER:',phone_number,'\n')
        
#     elif column_specific.shape[0] == 1:
        
#         phone_number = column_specific.iloc[0].phone
#         print('PHONE_NUMBER:',phone_number,'\n')
        
#     else:
#         # extract addresses to filter out based on frequency for golden record
#         phn_lst = column_specific['phone'].tolist()
# #         print("phn_lst:",phn_lst,'\n')

#         #Frequency counts of each address in list
#         phone_counts = {x:phn_lst.count(x) for x in phn_lst}
# #         print("phone_counts:",phone_counts,":: set counts:",set(phone_counts.values()),'\n')
        
#         if len(phone_counts) == 1:
#             phone_number = list(phone_counts.keys())[0]
#             print('PHONE_NUMBER:',phone_number,'\n')
        
#         else:
        
#             # Extract the most recent record of the above dataframe
#             column_specific['date_created'] = pd.to_datetime(column_specific['date_created'],errors = 'coerce')
# #             print(type(column_specific['date_created']))

#             if column_specific['date_created'].isnull().values.any() == True:
#                 phone_number = column_specific['phone'].values.tolist()
#                 phone_number = (phone_number[-1] if len(phone_number) > 1 else phone_number)
#                 print('PHONE_NUMBER:',phone_number,'\n')
                
#             else:
                
#                 # Extract the most recent record of the above dataframe
#                 date_created = max(column_specific['date_created'].tolist())

#                 phone_number = column_specific['phone'][(column_specific['date_created']==date_created)].values.tolist()
#                 print('PHONE_NUMBER:',phone_number,'\n')
                

# #     print("--------")
    
#     insrt_cmd = "INSERT INTO mct(customer_id,first_name,last_name,address_line1,address_line2,city,state,country,zip,phone) VALUES (1,'"+str(first_name)+"','"+str(last_name)+"','"+str(adl1)+"','"+str(adl2)+"','"+str(city)+"','"+str(state)+"','"+str(country)+"','"+str(zip)+"','"+str(phone_number)+"')" 
#     print(insrt_cmd)
# #     pgcursor.execute(insrt_cmd)
    
    print("---------------------------------------------------------------")


In [ ]:
# insrt_cmd1 = "INSERT INTO stg_BLC_CUSTOMER(first_name,last_name,address_line1,address_line2,city,state,country,zip,phone) " \
#             "VALUES ('"+first_name+"','"+last_name+"','"+address_line1+"','"+address_line2+"','"+city+"','"+state+"','"+country+"','"+zip+"','"+phone"')" 
# pgcursor.execute(insrt_cmd1)

In [ ]:
# pgcursor.execute(insrt_cmd1)


In [ ]:
# new_master["master_customer_id"] = range(1, 1+len(new_master))
# new_master

In [ ]:
# new_master.to_sql(name='master_customer_test', con=postgres_engine, if_exists='append', index=False, method='multi')